In [3]:
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('ohlc_1m_data_last_day.csv')

# Ensure the 'timestamp' column is parsed as datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set 'timestamp' as the index
df.set_index('timestamp', inplace=True)

# Calculate the Exponential Moving Average (EMA)
df['EMA'] = df['close'].ewm(span=20, adjust=False).mean()  # Adjust the span according to your requirements

# Resample the data to 4-hour intervals, using the last observed value
df_resampled = df.resample('4H').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'EMA': 'last'
}).dropna()

# Generate the signals based on EMA
df_resampled['signal'] = 0
df_resampled['EMA signal'] = ''

# Buy signal: When the close price crosses above the EMA
df_resampled.loc[df_resampled['close'] > df_resampled['EMA'], 'signal'] = 1
df_resampled.loc[df_resampled['close'] > df_resampled['EMA'], 'EMA signal'] = 'Buy'

# Sell signal: When the close price crosses below the EMA
df_resampled.loc[df_resampled['close'] < df_resampled['EMA'], 'signal'] = -1
df_resampled.loc[df_resampled['close'] < df_resampled['EMA'], 'EMA signal'] = 'Sell'

# Save the result to a CSV file
df_resampled.to_csv('EMA_4h_signal.csv')

# To view the first few rows of the DataFrame with the signals
print(df_resampled.head())


C:\Users\MTariq\AppData\Local\Temp\ipykernel_4352\3267446557.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('4H').agg({


                        open     high      low    close          EMA  signal  \
timestamp                                                                      
2020-01-01 00:00:00  7195.24  7245.00  7175.46  7225.01  7227.161154      -1   
2020-01-01 04:00:00  7225.00  7236.27  7199.11  7209.83  7210.428385      -1   
2020-01-01 08:00:00  7209.83  7237.73  7180.00  7197.20  7205.165829      -1   
2020-01-01 12:00:00  7197.20  7255.00  7196.15  7234.19  7230.579859       1   
2020-01-01 16:00:00  7234.20  7249.99  7214.00  7229.48  7230.260333      -1   

                    EMA signal  
timestamp                       
2020-01-01 00:00:00       Sell  
2020-01-01 04:00:00       Sell  
2020-01-01 08:00:00       Sell  
2020-01-01 12:00:00        Buy  
2020-01-01 16:00:00       Sell  
